In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df = train_df.drop(['id', 'timecc'], axis=1)
test_df = test_df.drop(['id', 'timecc'], axis=1)

In [2]:
# 创建交叉特征
train_df['kd_ratio'] = train_df['kills'] / (train_df['deaths'] + 1)
train_df['multikill_ratio'] = train_df['largestmultikill'] / (train_df['kills'] + 1)
train_df['heal_damage_ratio'] = train_df['totheal'] / (train_df['totdmgdealt'] + 1)
train_df['damage_taken_ratio'] = train_df['totdmgtochamp'] / (train_df['totdmgtaken'] + 1)
train_df['kill_turret_ratio'] = train_df['kills'] / (train_df['dmgtoturrets'] + 1)
train_df['crit_damage_ratio'] = train_df['largestcrit'] / (train_df['totdmgdealt'] + 1)

test_df['kd_ratio'] = test_df['kills'] / (test_df['deaths'] + 1)
test_df['multikill_ratio'] = test_df['largestmultikill'] / (test_df['kills'] + 1)
test_df['heal_damage_ratio'] = test_df['totheal'] / (test_df['totdmgdealt'] + 1)
test_df['damage_taken_ratio'] = test_df['totdmgtochamp'] / (test_df['totdmgtaken'] + 1)
test_df['kill_turret_ratio'] = test_df['kills'] / (test_df['dmgtoturrets'] + 1)
test_df['crit_damage_ratio'] = test_df['largestcrit'] / (test_df['totdmgdealt'] + 1)

In [3]:
from sklearn.model_selection import train_test_split

X = train_df.drop('win', axis=1)
y = train_df['win']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report


clf1 = RandomForestClassifier()
clf2 = LogisticRegression(max_iter=3000, n_jobs=8)
clf3 = GradientBoostingClassifier()

estimators = [("RandomForest",clf1),("Logistic",clf2),("GBDT",clf3)]
voting_clf = VotingClassifier(estimators, voting="soft")

# 训练投票分类器
voting_clf.fit(X_train, y_train)

# 在验证集上进行预测
y_pred = voting_clf.predict(X_val)

print(classification_report(y_val, y_pred))

/home/moguw/miniconda3/envs/paddle/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Number of positive: 81013, number of negative: 80987
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5699
[LightGBM] [Info] Number of data points in the train set: 162000, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500080 -> initscore=0.000321
[LightGBM] [Info] Start training from score 0.000321
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      9022
           1       0.85      0.85      0.85      8978

    accuracy                           0.85     18000
   macro avg       0.85      0.85      0.85     18000
weighted avg       0.85      0.85      0.85     18000



In [5]:
test_predict = voting_clf.predict(test_df)

In [7]:
test_predict

array([0, 0, 1, ..., 0, 0, 1])

In [8]:
pd.DataFrame({'win': test_predict
             }).to_csv('submission.csv', index=None)

In [9]:
import zipfile

# 定义文件名
csv_filename = 'submission.csv'
zip_filename = 'submission.zip'

# 创建一个Zip文件并将CSV文件添加进去
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

print(f'{csv_filename} has been compressed into {zip_filename}')

submission.csv has been compressed into submission.zip
